In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [2]:
df = pd.read_csv('C:/Users/toled/Documents/UnB/CE2 - Python/trabalho 2/StudentsPerformance.csv')
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [3]:
df['gender'].unique()

array(['female', 'male'], dtype=object)

In [4]:
df['race/ethnicity'].unique()

array(['group B', 'group C', 'group A', 'group D', 'group E'],
      dtype=object)

In [5]:
df['parental level of education'].unique()

array(["bachelor's degree", 'some college', "master's degree",
       "associate's degree", 'high school', 'some high school'],
      dtype=object)

In [6]:
df['lunch'].unique()

array(['standard', 'free/reduced'], dtype=object)

In [7]:
df['test preparation course'].unique()

array(['none', 'completed'], dtype=object)

In [8]:
df['reading score'].mean()

69.169

## Tentando modelar via regressão logistica se as notas em matemática, leitura e escrita explicam se fez curso preparatório ou não
(a interpretação real seria o contrário na realidade)

In [12]:
X = df[['math score', 'reading score', 'writing score']]
y = df['test preparation course']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

model = LogisticRegression()
model.fit(X_train, y_train)

predictions = model.predict(X_test)

print(classification_report(y_test, predictions))

confusion_matrix(y_test, predictions)

              precision    recall  f1-score   support

   completed       0.58      0.45      0.51       111
        none       0.71      0.81      0.76       189

    accuracy                           0.68       300
   macro avg       0.65      0.63      0.63       300
weighted avg       0.67      0.68      0.67       300



array([[ 50,  61],
       [ 36, 153]], dtype=int64)

### Não explicou tão bem para um conjunto tão pequeno

In [22]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder

X = df[['race/ethnicity', 'parental level of education', 'lunch', 'test preparation course', 'math score', 'reading score', 'writing score']]
y = df['gender']

encoder = OneHotEncoder()
X_encoded = encoder.fit_transform(X)

model = RandomForestClassifier()
model.fit(X_encoded, y)

df['predictions'] = model.predict(X_encoded)

df

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score,predictions
0,female,group B,bachelor's degree,standard,none,72,72,74,female
1,female,group C,some college,standard,completed,69,90,88,female
2,female,group B,master's degree,standard,none,90,95,93,female
3,male,group A,associate's degree,free/reduced,none,47,57,44,male
4,male,group C,some college,standard,none,76,78,75,male
...,...,...,...,...,...,...,...,...,...
995,female,group E,master's degree,standard,completed,88,99,95,female
996,male,group C,high school,free/reduced,none,62,55,55,male
997,female,group C,high school,free/reduced,completed,59,71,65,female
998,female,group D,some college,standard,completed,68,78,77,female


# Podemos ver que o modelo sofreu de overfitting. Ou seja, explicou 100% os dados, mas talvez não performasse bem com uma nova observação.

In [30]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder

X = df[['test preparation course', 'math score', 'reading score', 'writing score']]
y = df['gender']

encoder = OneHotEncoder()
X_encoded = encoder.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.3)

model = RandomForestClassifier()
model.fit(X_train, y_train)

predictions = model.predict(X_test)

print(classification_report(y_test, predictions))

confusion_matrix(y_test, predictions)

              precision    recall  f1-score   support

      female       0.58      0.72      0.64       146
        male       0.65      0.50      0.57       154

    accuracy                           0.61       300
   macro avg       0.61      0.61      0.60       300
weighted avg       0.62      0.61      0.60       300



array([[105,  41],
       [ 77,  77]], dtype=int64)

Vemos aqui que separando o conjunto em treino e teste, a matriz de confusão de altera significativamente. Em geral, a regressão logística performou melhor.